# A look at the results of the rest of SMK images person/people classification AMT batches

On 2020-01-20 we ran 7 more batches of images in the *SMK does image contain a person or people? with thumbnails and less pay* project, using as data the from `datasources.Smk.get_sample('the-rest-after-next5000-not-on-iip-smk-dk')`. This is the rest of the Open Data SMK images in the Public Domain, excluding the first 1000 and the next 5000 images, and additionally filtering out images which SMK serves from the server *iip.smk.dk* because it was causing issues earlier.

In [ ]:
import pandas as pd
import altair as alt
from datasources import Smk

In [ ]:
smk = Smk()

In [ ]:
%time
sample = smk.get_sample('the-rest-after-next5000-not-on-iip-smk-dk')
sample.shape

In [ ]:
amt_result_dir = 'data/amt_results/the-rest-of-smk-excluding-iip-smk-server/'
amt_result_files = ['Batch_3895760_batch_results.csv',
                    'Batch_3895771_batch_results.csv',
                    'Batch_3895772_batch_results.csv',
                    'Batch_3895786_batch_results.csv',
                    'Batch_3895787_batch_results.csv',
                    'Batch_3895789_batch_results.csv',
                    'Batch_3895791_batch_results.csv']

In [ ]:
amt_results = pd.read_csv(f"{amt_result_dir}/{amt_result_files[0]}")

In [ ]:
%time
amt_results = pd.DataFrame()
for file in amt_result_files:
    amt_results = amt_results.append(pd.read_csv(f"{amt_result_dir}/{file}", index_col='HITId'))

amt_results.shape

In [ ]:
amt_results.columns

Whoop Altair is going to have issues if column names have dots in them; this is a JSON issue on Vega-Lite side of things as the dot is for objects. So rename them.

In [ ]:
amt_results = amt_results.rename(columns=lambda l: l.replace('.', '_'))

Might be a good idea also to rename the hyphens. Anyway, data parsing looks ready.

In [ ]:
amt_results.head(2)

### Sanity checks on the data

The project has 3 assignments per HIT, so there should be three lines per image in the sample. These could go to our tests module.

In [ ]:
assert len(sample) * 3 == len(amt_results), "Should have as many images as assignments."
assert len(sample) == amt_results.index.unique().size, "Should have as many images as unique HITs"
assert (amt_results.groupby('Input_id').size() == 3).all(), "Should have 3 assignments for each image."
assert sample.index.difference(amt_results.set_index('Input_id').index).empty, "Should have empty intersection of sample ids and HIT ids "

# Results summary

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
alt.Chart(amt_results).mark_bar().encode(
    x='count()',
    y='Answer_image-contains_label',
)

What about annotation agreement?

First let's make a numeric, binomial 1/0 variable out of the Yes' and Nos, which we can also use for calculations.

In [ ]:
amt_results['Answer_has-people'] = amt_results['Answer_image-contains_label'].replace({'Yes': 1, 'No': 0})

In [ ]:
alt.Chart(amt_results.groupby('Input_id')['Answer_has-people'].mean().reset_index(), width=100, height=200).mark_bar(width=15).encode(
    x=alt.X('Answer_has-people', scale=alt.Scale(domain=[0, 1])),
    y='count()'
)

Good, it is well out of uniform distribution.

In [ ]:
alt.Chart(amt_results, width=700).transform_density(
    'WorkTimeInSeconds',
    as_=['WorkTimeInSeconds', 'density']
).mark_line(opacity=0.6).encode(
    x='WorkTimeInSeconds:Q',
    y='density:Q'
)